# IMDB Investigation

### Import Library

In [1]:
# import
import numpy as np
import pandas as pd


### Import Data

In [2]:
basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

### Review Dataframes

In [3]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [5]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1976
1,tt0000002,5.8,264
2,tt0000003,6.5,1825
3,tt0000004,5.6,178
4,tt0000005,6.2,2617


### Preprocessing Cleanup

#### Title Basics

##### Find NAN Values

In [6]:
basics = basics.replace({'\\N':np.nan})

In [ ]:
### Evaluating NAN values in columns

In [ ]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1334522
endYear           9783050
runtimeMinutes    6969959
genres             444668
dtype: int64

In [8]:
basics.shape

(9890228, 9)

In [9]:
basics.dtypes

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object

In [11]:
basics = basics.astype({"tconst":"string","titleType":"string","primaryTitle":"string","originalTitle":"string", "isAdult":"int", "startYear":"int", "endYear": "int","runtimeMinutes":"int", "genres":"string"})

ValueError: cannot convert float NaN to integer

In [ ]:
basics.dtypes

In [13]:
basics.isna().sum()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...
9890223,False,False,False,False,False,False,True,True,False
9890224,False,False,False,False,False,False,True,True,False
9890225,False,False,False,False,False,False,True,True,False
9890226,False,False,False,False,False,False,True,False,False


##### Eliminate movies that are null for runtimeMinutes

In [ ]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [ ]:
basics.isna().sum()

##### Eliminate movies that are null for genre

In [ ]:
basics.dropna(subset=['genres'], inplace=True)

In [ ]:
df_b_save = basics.copy()

##### keep only titleType==movie

In [ ]:
basics = basics[basics['titleType'] == 'movie']

In [ ]:
basics.head()

In [ ]:
print(basics['startYear'].unique())

In [ ]:
basics['startYear'].type()

##### keep startYear 2000-2022

In [ ]:
basics.dropna(subset=['startYear'], inplace=True)

In [ ]:
basics = basics[(basics['startYear'] >= 2000) & (df['startYear'] <= 2022)]

In [ ]:
##### Eliminate movies that include "Documentary" in genre